# Week 1 study case.
## Study requirements.
USE SEAL!!!!!
## Participants.
1. Data providers - individuals (alba-saeng).
2. Client (The government).
3. Service provider (A surveying company).
## Scenario.
0.  The client has to decide
    *  The level of security (Probably bound be the law).
    *  The number of samples (Probably bound by the price of the service).
1.  The client asks for a service to calculate the __average__ age of data providers to the service provider.
2.  The service provider hands over the __*key generating program*__ to generate
    *  a public key.
    *  a set of evaluation keys.
    *  an appropriate homomorphic encryption parameters. (encryption policy)
3.  The client generates the necessary requirements, using the  __*key generating program*__, together with the desired number of samples, and remits the public key, the evaluation key and the number of samples to the service provider. The client holds the private key and does not reveal it.
4.  The service provider calls the data providers and distributes the __*encrypting program*__ that contains embedded public keys (the service provider holds the list of data providers).
5.  Data providers encrypt their ages using the __*encrypting program*__ and submit the encrypted data to the service provider.
6.  The service provider calculates the __average__ age in encrypted domain, then return the result to the client.
7.  The client decrypts the result with the private key the party holds to itself.

In [34]:
import numpy as np
import tenseal as ts

In [119]:
"""
x is the input fixed number which is of integer datatype
e is the number of fractional bits for example in Q1.15 e = 15
"""
def to_float(x,e):
    c = abs(x)
    sign = 1
    if x < 0:
        # convert back from two's complement
        c = x - 1
        c = ~c
        sign = -1
    f = (1.0 * c) / (2 ** e)
    f = f * sign
    return f
"""
f is the input floating point number
e is the number of fractional bits in the Q format.
    Example in Q1.15 format e = 15
"""
def to_fixed(f,e):
    a = f* (2**e)
    b = int(round(a))
    if a < 0:
        # next three lines turns b into it's 2's complement.
        b = abs(b)
        b = ~b
        b = b + 1
    return b

def to_fixed_array(float_array, e):
    return list(map(lambda x: to_fixed(x, e), float_array))

def to_float_array(fixed_array, e):
    return list(map(lambda x: to_float(x, e), fixed_array))

def trunc(fp, e):
    return fp >> e


def trunc_array(fp_array, e):
    return list(map(lambda x: x >> e, fp_array))


def test_array(fp_array, e):
    return list(map(lambda x: x + e, fp_array))

In [62]:
a = to_fixed(1.5, 15)
b = to_fixed(2.0, 15)
ab = a * b
ab

3221225472

In [64]:
trunc_ab = trunc(ab, 15)
float_ab = to_float(trunc_ab, 15)
print(trunc_ab, float_ab)

98304 3.0


In [60]:
f = test_array(s, 15)
s

[6, 5, 11, 12, 15]

In [38]:
FRACTIONAL_BITS = 15
# Client
security_level = {
    'scheme_type': ts.SCHEME_TYPE.BFV,
    'poly_modulus_degree': 4096,
    'plain_modulus': 1032193
}

In [65]:
context = ts.context(security_level['scheme_type'], poly_modulus_degree=security_level['poly_modulus_degree'], plain_modulus=security_level['plain_modulus'])
context

In [85]:
n_count = 5
plain_vector = np.random.randint(10, 80, size=n_count).tolist()
print('plain vector:', plain_vector)
print('plain average:', np.average(plain_vector))

plain vector: [32, 57, 17, 44, 37]
plain average: 37.4


In [137]:
# test in plain text
dividend = [1/n_count]*n_count
fp_v = to_fixed_array(plain_vector, FRACTIONAL_BITS)

fp_d = to_fixed_array(dividend, FRACTIONAL_BITS)
print(fp_v, fp_d)
print('add', fp_v + fp_d)
mul_result = fp_v[0] * fp_d[0]
result = trunc(mul_result, FRACTIONAL_BITS)
result = to_float(result, FRACTIONAL_BITS)
print(result, mul_result, plain_vector[0] * dividend[0])

[1048576, 1867776, 557056, 1441792, 1212416] [6554, 6554, 6554, 6554, 6554]
add [1048576, 1867776, 557056, 1441792, 1212416, 6554, 6554, 6554, 6554, 6554]
6.400390625 6872367104 6.4


In [135]:
fp_plain_vector = to_fixed_array(plain_vector, FRACTIONAL_BITS)

encrypted_vector = ts.bfv_vector(context, fp_plain_vector)
# encrypted_vector = ts.bfv_vector(context, plain_vector)

dividend = [1/n_count]*n_count
fp_d = to_fixed_array(dividend, FRACTIONAL_BITS)

fp_encrypted_dividend = ts.bfv_vector(context, fp_d)
print(plain_vector, dividend)
print(fp_plain_vector, fp_d)
print(to_float_array(fp_plain_vector, FRACTIONAL_BITS), to_float_array(fp_d, FRACTIONAL_BITS))

[32, 57, 17, 44, 37] [0.2, 0.2, 0.2, 0.2, 0.2]
[1048576, 1867776, 557056, 1441792, 1212416] [6554, 6554, 6554, 6554, 6554]
[32.0, 57.0, 17.0, 44.0, 37.0] [0.20001220703125, 0.20001220703125, 0.20001220703125, 0.20001220703125, 0.20001220703125]


In [136]:
enc_result = encrypted_vector + fp_encrypted_dividend
dec_result = enc_result.decrypt()

print(dec_result)

[22937, -190056, -468583, 416153, 186777]


In [94]:
def encrypted_sum(ev):
    d = ev.decrypt()
    print(d)

encrypted_sum(encrypted_vector)

[32, 57, 17, 44, 37]


In [87]:
result = encrypted_vector * fp_encrypted_dividend
result

In [88]:
mul_result = result.decrypt()

trunc_mul_result = trunc_array(mul_result, FRACTIONAL_BITS)
print(mul_result, trunc_mul_result)
# k = to_float(s[0], FRACTIONAL_BITS)
# k

[26110, -405076, 78383, -222147, 352750] [0, -13, 2, -7, 10]


In [44]:
ns = np.sum(s)
fns = to_float(ns, FRACTIONAL_BITS)
fns

0.001495361328125

In [45]:
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
public_context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
sk = public_context.secret_key()
public_context.make_context_public()


In [46]:
add_result = encrypted_vector + [1, 2, 3, 4, 5]
print(add_result.decrypt())

[35, 31, 62, 67, 82]
